# Heuristic 평가

Heuristic 평가는 불충분한 시간이나 정보로 인해 완벽하게 합리적인 판단을 할 수 없을 때, 빠르고 간편하게 사용할 수 있는 추론 방법입니다.

(이는 LLM as Judge 를 활용할 때 드는 시간과 비용을 절약할 수 있다는 강점을 가지고 있기도 합니다.)



(참고) 아래의 코드 주석을 해제하여 라이브러리를 업데이트 후 진행합니다.

In [24]:
# !pip install -qU langsmith langchain-teddynote rouge-score

In [1]:
!free -h

               total        used        free      shared  buff/cache   available
Mem:           7.7Gi       1.2Gi       6.3Gi       2.0Mi       281Mi       6.3Gi
Swap:          2.0Gi       1.1Gi       958Mi


In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [10]:
# watch -n 2 free -h : 터미널에 입력해서 모니터링 ㄱㄱ

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# # !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("law_rag_eval")

LangSmith 추적을 시작합니다.
[프로젝트명]
law_rag_eval


In [5]:
import sys
sys.path.append("/home/jun/my_project/Law_RAG_PJ")

In [6]:
!pwd

/home/jun/my_project/Law_RAG_PJ/rag_eval


## RAG 성능 테스트를 위한 함수 정의

테스트에 활용할 RAG 시스템을 생성하겠습니다.

In [3]:
from law_rag import PINECONERAG
from langchain_groq import ChatGroq

# CHROMARAG 객체 생성
rag = PINECONERAG(ChatGroq(model="gemma2-9b-it",temperature=0), # gemma2-9b-it(평가용) , llama-3.2-11b-text-preview(연습용)
)

# 검색기(retriever) 생성
retriever = rag.create_retriever()

# 체인(chain) 생성
chain = rag.create_chain(retriever)

# 질문에 대한 답변 생성
# chain.invoke("임차권등기명령은 어떻게 해?")

[init_pinecone_index]
{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {'law-pdf-hypersearch_rag': {'vector_count': 2139}},
 'total_vector_count': 2139}


In [13]:
chain.invoke("보증금 5천만원에 임차한 집에서 8개월째 살고 있습니다. 집주인이 주변시세가 많이 올랐다며, 보증금을 천만원을 더 올려달라고 합니다. 보증금을 올려주어야 하나요?")

'저는 법률 상담자입니다. 주택임대차보호법에 따라 임차인의 권리와 의무를 설명드리겠습니다.\n\n제7조의2에 따르면, 임차인은 약정한 보증금이 임차주택에 관한 조세, 공과금, 그 밖의 부담의 증감이나 경제사정의 변동으로 인하여 적절하지 아니하게 된 때에는 장래에 대하여 그 증감을 청구할 수 있습니다. 그러나 증액청구는 임대차계약 또는 약정한 보증금의 증액이 있은 후 1년 이내에는 하지 못합니다.\n\n또한, 제7조의2에 따르면, 증액청구는 약정한 보증금의 20분의 1의 금액을 초과하지 못합니다. 다만, 특별시ㆍ광역시ㆍ특별자치시ㆍ도 및 특별자치도는 관할 구역 내의 지역별 임대차 시장 여건 등을 고려하여 본문의 범위에서 증액청구의 상한을 조례로 달리 정할 수 있습니다.\n\n따라서, 집주인이 주변시세가 많이 올랐다며, 보증금을 천만원을 더 올려달라고 하는 경우, 임차인은 증액청구를 할 수 있지만, 증액청구는 임대차계약 또는 약정한 보증금의 증액이 있은 후 1년 이내에는 하지 못하고, 약정한 보증금의 20분의 1의 금액을 초과하지 못해야 합니다.\n\n따라서, 임차인은 보증금을 올려주어야 하나, 증액청구는 임대차계약 또는 약정한 보증금의 증액이 있은 후 1년 이내에는 하지 못하고, 약정한 보증금의 20분의 1의 금액을 초과하지 못해야 합니다.'

In [14]:
chain.invoke("임차한 집에 거주하던 중 제 돈으로 집수리를 했습니다. 나중에 돌려받을 수 있나요?")

'임차인으로서 집수리를 위해 제 돈을 사용한 경우, 임차인에게 보증금의 회수권이 있습니다. 주택임대차보호법 제3조의2에 따라 임차인이 임차주택에 대한 보증금을 회수할 수 있습니다.\n\n임차인이 임차주택에 대한 보증금을 회수하려면, 임차인이 임차주택에 대한 보증금을 지급한 사실을 증명해야 합니다. 또한, 임차인이 임차주택에 대한 보증금을 회수하려면, 임차인이 임차주택에 대한 보증금을 회수할 권리가 있는지 여부를 확인해야 합니다.\n\n임차인이 임차주택에 대한 보증금을 회수할 권리가 있는지 여부는, 임차인이 임차주택에 대한 보증금을 지급한 사실을 증명하고, 임차인이 임차주택에 대한 보증금을 회수할 권리가 있는지 여부를 확인하는 과정을 통해 확인할 수 있습니다.\n\n임차인이 임차주택에 대한 보증금을 회수할 권리가 있는 경우, 임차인은 임차주택에 대한 보증금을 회수할 수 있습니다. 그러나, 임차인이 임차주택에 대한 보증금을 회수할 권리가 없는 경우, 임차인은 임차주택에 대한 보증금을 회수할 수 없습니다.\n\n따라서, 임차인이 임차주택에 대한 보증금을 회수하려면, 임차인이 임차주택에 대한 보증금을 지급한 사실을 증명하고, 임차인이 임차주택에 대한 보증금을 회수할 권리가 있는지 여부를 확인해야 합니다.'

In [15]:
chain.invoke("임차하여 살고 있는 집이 경매로 넘어갔다고 합니다. 보증금을 돌려받을 수 있을까요?")

'임차주택이 경매로 넘어간 경우, 보증금을 돌려받을 수 있는지에 대한 문제입니다. \n\n임차주택이 경매로 넘어간 경우, 보증금을 돌려받을 수 있는지에 대한 규정은 주택임대차보호법 제3조의2에 명시되어 있습니다. \n\n제3조의2(보증금의 회수) ① 임차인이 임차주택에 대하여 보증금을 반환받지 못한 경우, 임차인은 임차주택의 소재지를 관할하는 지방법원, 지방법원지원 또는 시·군 법원에 임차권등기명령을 신청할 수 있습니다. \n\n따라서, 임차주택이 경매로 넘어간 경우, 보증금을 돌려받을 수 있는지에 대한 여부는 임차권등기명령의 신청 여부에 따라 결정됩니다. \n\n임차권등기명령을 신청하면, 임차주택의 소유권이 경매로 넘어간 경우에도 보증금을 돌려받을 수 있습니다. \n\n그러나, 임차권등기명령을 신청하지 않은 경우, 보증금을 돌려받을 수 없습니다. \n\n따라서, 임차주택이 경매로 넘어간 경우, 보증금을 돌려받을 수 있는지에 대한 여부는 임차권등기명령의 신청 여부에 따라 결정됩니다. \n\n이러한 규정은 주택임대차보호법 제3조의2에 명시되어 있습니다. \n\n따라서, 임차주택이 경매로 넘어간 경우, 보증금을 돌려받을 수 있는지에 대한 여부는 임차권등기명령의 신청 여부에 따라 결정됩니다. \n\n이러한 규정은 주택임대차보호법 제3조의2에 명시되어 있습니다. \n\n따라서, 임차주택이 경매로 넘어간 경우, 보증금을 돌려받을 수 있는지에 대한 여부는 임차권등기명령의 신청 여부에 따라 결정됩니다. \n\n이러한 규정은 주택임대차보호법 제3조의2에 명시되어 있습니다. \n\n따라서, 임차주택이 경매로 넘어간 경우, 보증금을 돌려받을 수 있는지에 대한 여부는 임차권등기명령의 신청 여부에 따라 결정됩니다. \n\n이러한 규정은 주택임대차보호법 제3조의2에 명시되어 있습니다. \n\n따라서, 임차주택이 경매로 넘어간 경우, 보증금을 돌려받을 수 있는지에 대한 여부는 임차권등기명령의 신청 여부에 따라 결정됩니다. \n\n이러한 규정은 주택임대차보호법 제3조의2에 명시되어 있습

In [16]:
chain.invoke("보일러가 고장이 났는데 집주인은 임차인인 제가 고치는 것이라고 하여 어쩔 수 없이 제 돈으로 보일러를 고쳤는데, 이런 경우에도 임차인이 고쳐야 하나요?")

'제가 법률 상담자로서 답변드리겠습니다.\n\n보일러가 고장이 났을 때, 집주인은 임차인인 당신에게 고치라고 하여 어쩔 수 없이 당신이 돈을 들여 고쳤을 경우, 임차인이 고쳐야 하는지 여부는 민법 제626조에 따라 결정됩니다.\n\n민법 제626조 제1항은 "임차인이 임차물의 보존에 관한 필요비를 지출한 때에는 임대인에 대하여 그 상환을 청구할 수 있다."고 규정하고 있습니다. 이 규정은 임차인이 임차물의 보존에 관한 필요비를 지출한 경우, 임대인에게 그 상환을 청구할 수 있는 권리를 부여합니다.\n\n그러나, 보일러의 고장에 대한 책임은 집주인에게 있으므로, 당신은 집주인에게 보일러의 고장에 대한 책임을 청구할 수 있습니다. 또한, 당신은 보일러를 고치기 위해 지출한 비용을 집주인에게 청구할 수 있습니다.\n\n따라서, 보일러가 고장이 났을 때, 집주인은 임차인인 당신에게 고치라고 하여 어쩔 수 없이 당신이 돈을 들여 고쳤을 경우, 임차인이 고쳐야 하는 것은 아니며, 집주인에게 책임을 청구할 수 있습니다.\n\n하지만, 집주인이 임차인인 당신에게 고치라고 하여 어쩔 수 없이 당신이 돈을 들여 고쳤을 경우, 임차인이 고쳐야 하는 경우도 있습니다. 예를 들어, 집주인이 임차인인 당신에게 고치라고 하여 어쩔 수 없이 당신이 돈을 들여 고쳤을 때, 집주인이 그 사실을 알지 못한 경우, 또는 집주인이 그 사실을 알았지만 고치지 않은 경우, 임차인이 고쳐야 합니다.\n\n따라서, 보일러가 고장이 났을 때, 집주인은 임차인인 당신에게 고치라고 하여 어쩔 수 없이 당신이 돈을 들여 고쳤을 경우, 임차인이 고쳐야 하는지 여부는 민법 제626조와 관련된 법적 규정에 따라 결정됩니다.'

`ask_question` 이라는 이름으로 함수를 생성합니다. 입력으로는 `inputs` 라는 딕셔너리를 받고, 출력으로는 `answer` 라는 딕셔너리를 반환합니다.

In [4]:
# 질문에 대한 답변하는 함수를 생성
def ask_question(inputs: dict):
    return {"answer": chain.invoke(inputs["question"])}

## 한글 형태소 분석기의 활용

한글 형태소 분석기는 한국어 문장을 가장 작은 의미 단위인 형태소로 분리하고 각 형태소의 품사를 판별하는 도구입니다. 

형태소 분석기의 주요 기능
- 문장을 형태소 단위로 분리
- 각 형태소의 품사 태깅
- 형태소의 기본형 추출

Kiwipiepy 라이브러리를 활용하여 한글 형태소 분석기를 사용할 수 있습니다.

In [6]:
from langchain_teddynote.community.kiwi_tokenizer import KiwiTokenizer

# 토크나이저 선언
kiwi_tokenizer = KiwiTokenizer()

sent1 = "안녕하세요. 반갑습니다. 내 이름은 테디입니다."
sent2 = "안녕하세용 반갑습니다~^^ 내 이름은 테디입니다!!"

# 토큰화
print(sent1.split())
print(sent2.split())

print("===" * 20)

# 토큰화
print(kiwi_tokenizer.tokenize(sent1))
print(kiwi_tokenizer.tokenize(sent2))

['안녕하세요.', '반갑습니다.', '내', '이름은', '테디입니다.']
['안녕하세용', '반갑습니다~^^', '내', '이름은', '테디입니다!!']
['안녕', '하', '세요', '.', '반갑', '습니다', '.', '나', '의', '이름', '은', '테디', '이', 'ᆸ니다', '.']
['안녕', '하', '세요', 'ᆼ', '반갑', '습니다', '~', '^^', '나', '의', '이름', '은', '테디', '이', 'ᆸ니다', '!!']


In [5]:
from langchain_teddynote.community.kiwi_tokenizer import KiwiTokenizer

# 토크나이저 선언
kiwi_tokenizer = KiwiTokenizer()

## Rouge (Recall-Oriented Understudy for Gisting Evaluation) 스코어

- 자동 요약 및 기계 번역의 품질을 평가하는 데 사용되는 평가지표 입니다.
- 생성된 텍스트가 참조 텍스트의 중요 키워드를 얼마나 포함하는지 측정합니다.
- n-gram 중첩을 기반으로 계산됩니다

> 참고: N-gram 이란?

![](./assets/n-gram.png)

**Rouge-1**
- 단어 단위의 유사도를 측정합니다.
- 두 문장간의 개별 단어 일치도를 평가합니다.

**Rouge-2**
- 두 단어 연속(bigram)의 중복 단위의 유사도를 측정합니다.
- 두 문장간의 연속된 두 단어 일치도를 평가합니다.
  
**Rouge-L**
- 최장 공통 부분 수열(Longest Common Subsequence, LCS)을 기반으로 한 유사도를 측정합니다.
- 문장 수준의 단어 순서를 고려하며, 연속적인 일치를 요구하지 않습니다
- 더 유연한 평가가 가능하며, 문장 구조의 유사성을 자연스럽게 반영합니다.

**예시**

예시 문장
- 원본: "오늘 아침 공원에서 조깅을 하다가 귀여운 강아지를 만났습니다."
- 생성: "오늘 아침 공원에서 산책을 하던 중 작고 귀여운 고양이를 보았습니다."

1. ROUGE-1
   - 각 단어를 개별적으로 비교합니다.
   - 일치하는 단어: "오늘", "아침", "공원에서", "귀여운"
   - 이 단어들이 양쪽 문장에 모두 나타나므로 점수에 반영됩니다.

2. ROUGE-2
   - 연속된 두 단어씩 비교합니다.
   - 일치하는 구: "오늘 아침", "아침 공원에서"
   - 이 두 단어 조합들이 양쪽 문장에 모두 나타나므로 점수에 반영됩니다.

3. ROUGE-L
   - 순서를 유지하면서 가장 긴 공통 부분을 찾습니다.
   - 가장 긴 공통 부분: "오늘 아침 공원에서 귀여운"
   - 이 단어들이 순서를 유지하면서 양쪽 문장에 모두 나타나므로, 이 시퀀스가 ROUGE-L 점수에 반영됩니다.

이 예시를 통해 각 ROUGE 지표의 특성을 더 명확히 볼 수 있습니다:
- ROUGE-1은 개별 단어의 일치를 중요시합니다.
- ROUGE-2는 더 긴 구의 일치를 찾아내어 문맥을 조금 더 고려합니다.
- ROUGE-L은 전체 문장 구조 내에서 가장 긴 공통 부분을 찾아 전반적인 유사성을 평가합니다.

In [7]:
from rouge_score import rouge_scorer

sent1 = "안녕하세요. 반갑습니다. 내 이름은 테디입니다."
sent2 = "안녕하세용 반갑습니다~^^ 내 이름은 테디입니다!!"
sent3 = "내 이름은 테디입니다. 안녕하세요. 반갑습니다."

scorer = rouge_scorer.RougeScorer(
    ["rouge1", "rouge2", "rougeL"], use_stemmer=False, tokenizer=KiwiTokenizer()
)

print(
    f"[1] {sent1}\n[2] {sent2}\n[rouge1] {scorer.score(sent1, sent2)['rouge1'].fmeasure:.5f}\n[rouge2] {scorer.score(sent1, sent2)['rouge2'].fmeasure:.5f}\n[rougeL] {scorer.score(sent1, sent2)['rougeL'].fmeasure:.5f}"
)
print("===" * 20)
print(
    f"[1] {sent1}\n[2] {sent3}\n[rouge1] {scorer.score(sent1, sent3)['rouge1'].fmeasure:.5f}\n[rouge2] {scorer.score(sent1, sent3)['rouge2'].fmeasure:.5f}\n[rougeL] {scorer.score(sent1, sent3)['rougeL'].fmeasure:.5f}"
)

[1] 안녕하세요. 반갑습니다. 내 이름은 테디입니다.
[2] 안녕하세용 반갑습니다~^^ 내 이름은 테디입니다!!
[rouge1] 0.77419
[rouge2] 0.62069
[rougeL] 0.77419
[1] 안녕하세요. 반갑습니다. 내 이름은 테디입니다.
[2] 내 이름은 테디입니다. 안녕하세요. 반갑습니다.
[rouge1] 1.00000
[rouge2] 0.92857
[rougeL] 0.53333


In [6]:
from rouge_score import rouge_scorer

## BLEU (Bilingual Evaluation Understudy) 스코어

주로 기계 번역 평가에 사용됩니다. 생성된 텍스트가 참조 텍스트와 얼마나 유사한지를 측정합니다.

n-gram 정밀도(precision)를 기반으로 계산됩니다

**BLEU의 주요 특징**

1. N-gram 정밀도
   - BLEU는 1-gram부터 4-gram까지의 정밀도를 계산합니다.
   - 각 n-gram의 정밀도는 생성된 텍스트에서 참조 텍스트와 일치하는 n-gram의 비율입니다.

2. 간결성 페널티(Brevity Penalty)
   - 생성된 텍스트가 참조 텍스트보다 짧을 경우 페널티를 부과합니다.
   - 이는 시스템이 짧은 문장만 생성하여 높은 정밀도를 얻는 것을 방지합니다.

3. 기하평균
   - 최종 BLEU 점수는 각 n-gram 정밀도의 기하평균에 간결성 페널티를 곱한 값입니다.

**예시**

- 원본: "오늘 아침에 공원에서 조깅을 하다가 귀여운 강아지를 만났습니다."
- 생성: "오늘 아침 공원에서 산책을 하던 중 작고 귀여운 고양이를 보았습니다."

1. 1-gram 정밀도
   - 일치하는 단어: "오늘", "아침", "공원에서", "귀여운"
   - 정밀도 = 4 / 10 = 0.4

2. 2-gram 정밀도
   - 일치하는 2-gram: "오늘 아침", "아침 공원에서"
   - 정밀도 = 2 / 9 ≈ 0.22

3. 3-gram, 4-gram 정밀도
   - 일치하는 3-gram, 4-gram 없음
   - 정밀도 = 0

4. 간결성 페널티
   - 두 문장의 길이가 비슷하므로 페널티 없음 (1.0)

5. 최종 BLEU 점수:
   - 기하평균(0.4, 0.22, 0, 0) * 1.0
   - 결과는 0에 가까운 낮은 점수가 됩니다.

**한계점**
- 의미를 고려하지 않고 단순 문자열 일치만 확인합니다.
- 단어의 중요도를 구분하지 않습니다.

BLEU 점수는 0에서 1 사이의 값을 가지며, 1에 가까울수록 더 높은 품질을 나타냅니다. 하지만 실제로는 완벽한 1점을 얻기가 매우 어렵습니다.

In [8]:
from nltk.translate.bleu_score import sentence_bleu

sent1 = "안녕하세요. 반갑습니다. 내 이름은 테디입니다."
sent2 = "안녕하세용 반갑습니다~^^ 내 이름은 테디입니다!!"
sent3 = "내 이름은 테디입니다. 안녕하세요. 반갑습니다."

# 토큰화
print(kiwi_tokenizer.tokenize(sent1, type="sentence"))
print(kiwi_tokenizer.tokenize(sent2, type="sentence"))
print(kiwi_tokenizer.tokenize(sent3, type="sentence"))

안녕 하 세요 . 반갑 습니다 . 나 의 이름 은 테디 이 ᆸ니다 .
안녕 하 세요 ᆼ 반갑 습니다 ~ ^^ 나 의 이름 은 테디 이 ᆸ니다 !!
나 의 이름 은 테디 이 ᆸ니다 . 안녕 하 세요 . 반갑 습니다 .


In [8]:
from nltk.translate.bleu_score import sentence_bleu

In [9]:
bleu_score = sentence_bleu(
    [kiwi_tokenizer.tokenize(sent1, type="sentence")],
    kiwi_tokenizer.tokenize(sent2, type="sentence"),
)
print(f"[1] {sent1}\n[2] {sent2}\n[score] {bleu_score:.5f}")
print("===" * 20)

bleu_score = sentence_bleu(
    [kiwi_tokenizer.tokenize(sent1, type="sentence")],
    kiwi_tokenizer.tokenize(sent3, type="sentence"),
)
print(f"[1] {sent1}\n[2] {sent3}\n[score] {bleu_score:.5f}")

[1] 안녕하세요. 반갑습니다. 내 이름은 테디입니다.
[2] 안녕하세용 반갑습니다~^^ 내 이름은 테디입니다!!
[score] 0.74879
[1] 안녕하세요. 반갑습니다. 내 이름은 테디입니다.
[2] 내 이름은 테디입니다. 안녕하세요. 반갑습니다.
[score] 0.95739


## METEOR 스코어

기계 번역의 품질을 평가하기 위해 개발된 평가지표 입니다.

**주요 특징**

1. 단어 매칭
   - 정확한 매칭: 동일한 단어
   - 어간 매칭: 같은 어근을 가진 단어 (예: "run"과 "running")
   - 동의어 매칭: 의미가 같은 단어 (예: "quick"과 "fast")
   - 의역 매칭: 구절 수준의 동의어 (주로 기계 번역에서 사용)

2. 정밀도와 재현율의 조화평균
   - 정밀도: 생성된 텍스트에서 참조 텍스트와 일치하는 단어의 비율
   - 재현율: 참조 텍스트에서 생성된 텍스트와 일치하는 단어의 비율
   - F-mean: 정밀도와 재현율의 조화평균

3. 순서 패널티
   - 일치하는 단어들의 순서가 얼마나 다른지 고려합니다.
   - 연속되지 않은 매칭에 대해 패널티를 부과합니다.

4. 가중치 부여
   - 각 매칭 유형(정확, 어간, 동의어, 의역)에 다른 가중치를 부여할 수 있습니다.

**METEOR 점수 계산 과정**

1. 단어 매칭: 모든 가능한 매칭을 찾습니다.
2. 정밀도와 재현율 계산: 매칭된 단어 수를 기반으로 계산합니다.
3. F-mean 계산: 정밀도와 재현율의 조화평균을 구합니다.
4. 순서 패널티 계산: 매칭된 단어들의 순서 차이를 고려합니다.
5. 최종 METEOR 점수: F-mean * (1 - 순서 패널티)

**예시**
- 참조: "The cat is on the mat"
- 생성: "On the mat is a cat"

1. 단어 매칭: 모든 단어가 매칭됩니다.
2. 정밀도 = 재현율 = 1 (모든 단어가 매칭되므로)
3. F-mean = 1
4. 순서 패널티: 순서가 다르므로 약간의 패널티 적용 (예: 0.1)
5. 최종 METEOR 점수 = 1 * (1 - 0.1) = 0.9

**METEOR의 장점**
1. 동의어와 어간 변형을 인식하여 더 유연한 평가가 가능합니다.
2. 정밀도와 재현율을 모두 고려합니다.
3. 단어 순서의 중요성을 반영합니다.
4. 단일 참조 텍스트에 대해서도 잘 작동합니다.

**METEOR vs BLEU vs ROUGE**
- METEOR는 단어의 의미적 유사성을 고려하여 더 유연한 평가가 가능합니다.
- BLEU보다 인간의 판단과 더 잘 일치하는 경향이 있습니다.
- ROUGE와 달리, 단어 순서를 명시적으로 고려합니다.
- METEOR는 계산이 더 복잡하고 시간이 오래 걸릴 수 있습니다.

In [6]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/jun/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
from nltk.corpus import wordnet as wn

wn.ensure_loaded()

In [12]:
from nltk.translate import meteor_score

sent1 = "안녕하세요. 반갑습니다. 내 이름은 테디입니다."
sent2 = "안녕하세용 반갑습니다~^^ 내 이름은 테디입니다!!"
sent3 = "내 이름은 테디입니다. 안녕하세요. 반갑습니다."

meteor = meteor_score.meteor_score(
    [kiwi_tokenizer.tokenize(sent1, type="list")],
    kiwi_tokenizer.tokenize(sent2, type="list"),
)

print(f"[1] {sent1}\n[2] {sent2}\n[score] {meteor:.5f}")
print("===" * 20)

meteor = meteor_score.meteor_score(
    [kiwi_tokenizer.tokenize(sent1, type="list")],
    kiwi_tokenizer.tokenize(sent3, type="list"),
)
print(f"[1] {sent1}\n[2] {sent3}\n[score] {meteor:.5f}")

[1] 안녕하세요. 반갑습니다. 내 이름은 테디입니다.
[2] 안녕하세용 반갑습니다~^^ 내 이름은 테디입니다!!
[score] 0.78849
[1] 안녕하세요. 반갑습니다. 내 이름은 테디입니다.
[2] 내 이름은 테디입니다. 안녕하세요. 반갑습니다.
[score] 0.96800


In [11]:
from nltk.translate import meteor_score

## SemScore

- [SEMSCORE: Automated Evaluation of Instruction-Tuned LLMs based on Semantic Textual Similarity](https://arxiv.org/pdf/2401.17072)

이 논문에서는 의미론적 텍스트 유사성(STS)을 사용하여 모델 출력을 황금 표준 응답과 직접 비교하는 SEMSCORE라는 간단하지만 효과적인 평가 지표를 제안합니다. 12개의 주요 지시 튜닝된 LLM의 출력을 8개의 널리 사용되는 텍스트 생성 평가 지표로 비교 평가한 결과, 제안된 SEMSCORE 지표가 인간 평가와의 상관관계 측면에서 다른 모든 평가 지표보다 우수한 성능을 보여주었습니다.

**SEMSCORE의 주요 특징**

1. 의미적 텍스트 유사성(Semantic Textual Similarity, STS) 기반
   - 생성된 텍스트와 참조 텍스트 간의 의미적 유사성을 측정합니다.
   - 단순한 단어 매칭을 넘어 문장의 전반적인 의미를 고려합니다.

2. 사전 훈련된 언어 모델 활용
   - BERT나 RoBERTa 같은 사전 훈련된 언어 모델을 사용하여 문장 임베딩을 생성합니다.
   - 이를 통해 문맥과 의미를 더 잘 포착할 수 있습니다.

3. 다중 참조 지원
   - 여러 개의 정답 참조를 고려할 수 있습니다.
   - 이는 특히 개방형 질문이나 창의적인 작업에서 유용합니다.

4. 세분화된 평가
   - 전체 응답뿐만 아니라 응답의 각 부분(예: 문장 단위)에 대해서도 평가할 수 있습니다.

5. 인간 평가와의 높은 상관관계
   - SEMSCORE는 인간 평가자들의 판단과 높은 상관관계를 보입니다.

**SEMSCORE의 계산 과정**

1. 텍스트 임베딩
   - 생성된 텍스트와 참조 텍스트를 사전 훈련된 언어 모델을 통해 벡터로 변환합니다.

2. 유사도 계산
   - 생성된 텍스트의 임베딩과 참조 텍스트의 임베딩 간의 코사인 유사도를 계산합니다.

3. 최대 유사도 선택
   - 여러 참조가 있는 경우, 가장 높은 유사도 점수를 선택합니다.

4. 정규화
   - 최종 점수를 0에서 1 사이의 값으로 정규화합니다.

**SEMSCORE의 장점**

1. 의미적 이해  
   - 표면적인 단어 일치를 넘어 문장의 의미를 고려합니다.

2. 유연성:
   - 다양한 형태의 정답을 허용하므로, 창의적인 작업이나 개방형 질문에 적합합니다.

3. 맥락 고려
   - 사전 훈련된 언어 모델을 사용하여 단어와 문장의 맥락을 더 잘 이해합니다.

4. 다국어 지원
   - 다국어 모델을 사용하면 여러 언어에 대해 평가할 수 있습니다.

**SEMSCORE vs 다른 평가 지표**
- BLEU, ROUGE와 달리 단순한 n-gram 매칭에 의존하지 않습니다.
- METEOR보다 더 고급화된 의미적 유사성을 측정합니다.
- BERTScore와 유사하지만, 지시사항 기반 작업에 특화되어 있습니다.

`SentenceTransformer` 모델을 사용하여 문장 임베딩을 생성하고, 두 문장 간의 코사인 유사도를 계산합니다.
- 논문에서 사용된 모델인 `all-mpnet-base-v2` 를 사용합니다.

In [11]:
from sentence_transformers import SentenceTransformer, util
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

sent1 = "안녕하세요. 반갑습니다. 내 이름은 테디입니다."
sent2 = "안녕하세용 반갑습니다~^^ 내 이름은 테디입니다!!"
sent3 = "내 이름은 테디입니다. 안녕하세요. 반갑습니다."

# SentenceTransformer 모델 로드
model = SentenceTransformer("all-mpnet-base-v2")

# 문장들을 인코딩
sent1_encoded = model.encode(sent1, convert_to_tensor=True)
sent2_encoded = model.encode(sent2, convert_to_tensor=True)
sent3_encoded = model.encode(sent3, convert_to_tensor=True)

# sent1과 sent2 사이의 코사인 유사도 계산
cosine_similarity = util.pytorch_cos_sim(sent1_encoded, sent2_encoded).item()
print(f"[1] {sent1}\n[2] {sent2}\n[score] {cosine_similarity:.5f}")

print("===" * 20)

# sent1과 sent3 사이의 코사인 유사도 계산
cosine_similarity = util.pytorch_cos_sim(sent1_encoded, sent3_encoded).item()
print(f"[1] {sent1}\n[2] {sent3}\n[score] {cosine_similarity:.5f}")

[1] 안녕하세요. 반갑습니다. 내 이름은 테디입니다.
[2] 안녕하세용 반갑습니다~^^ 내 이름은 테디입니다!!
[score] 0.86157
[1] 안녕하세요. 반갑습니다. 내 이름은 테디입니다.
[2] 내 이름은 테디입니다. 안녕하세요. 반갑습니다.
[score] 0.99191


위의 내용을 종합하여 정리한 Evaluator 는 다음과 같습니다.

In [8]:
from langsmith.schemas import Run, Example
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate import meteor_score
from sentence_transformers import SentenceTransformer, util
import os

# 토크나이저 병렬화 설정(HuggingFace 모델 사용)
os.environ["TOKENIZERS_PARALLELISM"] = "true"


def rouge_evaluator(metric: str = "rouge2") -> dict: # rouge1, rouge2, rougeL로 바꿀 수 있음
    # wrapper function 정의
    def _rouge_evaluator(run: Run, example: Example) -> dict:
        # 출력값과 정답 가져오기
        student_answer = run.outputs.get("answer", "")
        reference_answer = example.outputs.get("answer", "")

        # ROUGE 점수 계산
        scorer = rouge_scorer.RougeScorer(
            ["rouge1", "rouge2", "rougeL"], use_stemmer=True, tokenizer=KiwiTokenizer()
        )
        scores = scorer.score(reference_answer, student_answer)

        # ROUGE 점수 반환
        rouge = scores[metric].fmeasure

        return {"key": "ROUGE", "score": rouge}

    return _rouge_evaluator


def bleu_evaluator(run: Run, example: Example) -> dict:
    # 출력값과 정답 가져오기
    student_answer = run.outputs.get("answer", "")
    reference_answer = example.outputs.get("answer", "")

    # 토큰화
    reference_tokens = kiwi_tokenizer.tokenize(reference_answer, type="sentence")
    student_tokens = kiwi_tokenizer.tokenize(student_answer, type="sentence")

    # BLEU 점수 계산
    bleu_score = sentence_bleu([reference_tokens], student_tokens)

    return {"key": "BLEU", "score": bleu_score}


def meteor_evaluator(run: Run, example: Example) -> dict:
    # 출력값과 정답 가져오기
    student_answer = run.outputs.get("answer", "")
    reference_answer = example.outputs.get("answer", "")

    # 토큰화
    reference_tokens = kiwi_tokenizer.tokenize(reference_answer, type="list")
    student_tokens = kiwi_tokenizer.tokenize(student_answer, type="list")

    # METEOR 점수 계산
    meteor = meteor_score.meteor_score([reference_tokens], student_tokens)

    return {"key": "METEOR", "score": meteor}


def semscore_evaluator(run: Run, example: Example) -> dict:
    # 출력값과 정답 가져오기
    student_answer = run.outputs.get("answer", "")
    reference_answer = example.outputs.get("answer", "")

    # SentenceTransformer 모델 로드
    model = SentenceTransformer("all-mpnet-base-v2")

    # 문장 임베딩 생성
    student_embedding = model.encode(student_answer, convert_to_tensor=True)
    reference_embedding = model.encode(reference_answer, convert_to_tensor=True)

    # 코사인 유사도 계산
    cosine_similarity = util.pytorch_cos_sim(
        student_embedding, reference_embedding
    ).item()

    return {"key": "sem_score", "score": cosine_similarity}

Heuristic Evaluator 를 활용한 평가를 진행합니다.

In [9]:
from langsmith.evaluation import evaluate

# 평가자 정의
heuristic_evalulators = [
    rouge_evaluator(metric="rouge2"),
    # bleu_evaluator,
    # meteor_evaluator,
    semscore_evaluator,
]

# 데이터셋 이름 설정
dataset_name = "Law_RAG_Eval_Dataset"

# 실험 실행
experiment_results = evaluate(
    ask_question,
    data=dataset_name,
    evaluators=heuristic_evalulators,
    experiment_prefix="Heuristic-EVAL-pinecone-ensembleretriever-7:3",
    # 실험 메타데이터 지정
    metadata={
        "variant": "Heuristic-EVAL (Rouge, SemScore) 을 사용하여 평가",
    },
)

View the evaluation results for experiment: 'Heuristic-EVAL-pinecone-ensembleretriever-7:3-d7d16581' at:
https://smith.langchain.com/o/8fb13a5d-db0a-55ac-abc3-9ac5cc1b15ae/datasets/dec0eb44-d1f3-4cdf-aeea-6283fc0b7175/compare?selectedSessions=2c17c975-a91e-4512-a8f5-8e43f36a3664




0it [00:00, ?it/s]

/home/jun/.conda/envs/chromagroq/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


결과를 확인합니다.

![](./assets/eval-07.png)